In [1]:
%matplotlib qt
import numpy as np
import matplotlib.pyplot as plt
import os.path as op
from scipy.io import loadmat

In [2]:
data_dir = '/home/rob/data/MEG'
fname_fwd_model = 'gainConstrainedSub05.mat'
fname_inv_model = 'inverseConstrainedSub05.mat'

# $P = AA^{-1}$ matrix from BST results

In [3]:
fwd_model = loadmat(op.join(data_dir, fname_fwd_model))
A = fwd_model['Gain_constrained'][:306, :]
print('Shape of A =', np.shape(A))

inv_model = loadmat(op.join(data_dir, fname_inv_model))
A_inv = inv_model['ImagingKernel']
print('Shape of A_inv =', np.shape(A_inv))

P = A @ A_inv

plt.figure()
plt.imshow(P)
plt.colorbar()
plt.show()

Shape of A = (306, 15002)
Shape of A_inv = (15002, 306)


# $P = AA^{-1}$ matrix from solving with pseudo-inversion

In [4]:
U, S, VT = np.linalg.svd(A, full_matrices=0)
A_pinv = VT.T @ np.linalg.inv(np.diag(S)) @ U.T
print('Shape of A_pinv =', np.shape(A_pinv))

P_pseudo = A @ A_pinv

plt.figure()
plt.imshow(P_pseudo)
plt.colorbar()
plt.show()

Shape of A_pinv = (15002, 306)


# $P = AA^{-1}$ matrix from solving with Tikhonov regularization
$x = (A^{T} A + \phi I)^{-1} A^{T} b$

$A^{-1} = V \frac{\Lambda}{\Lambda^2 + \phi I} U^{T}$

In [5]:
phi = 1 / 3 ** 2
n, _ = np.shape(A)

U, L, VT = np.linalg.svd(A, full_matrices=0)

A_inv_TR = VT.T @ np.diag([l / (l ** 2 + phi) for l in L]) @ U.T
print('Shape of A_inv_TR =', np.shape(A_inv_TR))

P_TR = A @ A_inv_TR

fig = plt.figure()
ax1 = fig.add_subplot(121)
plt.imshow(P)
plt.colorbar()
ax2 = fig.add_subplot(122)
plt.imshow(P_TR)
plt.colorbar()
plt.show()

Shape of A_inv_TR = (15002, 306)


In [6]:
plt.figure()
plt.plot(np.arange(1,n+1), np.log(L), '-o')
plt.xlabel('i', fontsize=14)
plt.ylabel('ln($\sigma_i$)', fontsize=14)
plt.show()